**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

**SYNTAX:**
You can enter `→` by writing `->` and pressing TAB; pressing TAB again goes back to `->`. Similar useful combinations are:

| base form | alternate form |
|:---------:|:--------------:|
| ->  | → |
| \   | λ |
| neg | ¬ |
| top | ⊤ |
| bot | ⊥ |
| /\  | ∧ |
| \\/ | ∨ |
| <   | ⟨ |
| >   | ⟩ |
| forall | ∀ |
| exists | ∃ |
| Pi  | Π |
| Sigma | Σ |

<!---
| phi | |
| psi | |
-->

<!---

**PRO TRICKS:**
- You can ask Agda to provide the type of a closed expression. Just put the cursor nearby an expression and press SHIFT+TAB.
- You can ask Agda to compute the normal form of an expression enclosed in parantheses, such as `(λ x → x)` (already in normal form).

-->

<!--- Starred exercises "**Exercise***" should be skipped on a first reading and dealt with only after all the non-starred one are resolved. -->

# Curry-Howard isomorphism

| Logic    | Programming |     |
|----------|-------------|-----|
| theorems | types       | $A$ |
| proofs   | programs (terms) | $a$ |
| proof simplification | program execution | run $a$ |
| proof development | programming | write $a$ |
| proof checking | type checking | given $a, A$, does $a : A$ hold? |
| validity | type inhabitation | given $A$, find $a$ s.t. $a : A$ |
| ?        | type reconstruction | given $a$, find $A$ s.t. $a : A$ |

<!--- One important difference is that,
while in logic it is enough to have any proof $a$ of a theorem $A$
(in this sense all proofs of $A$ are the same),
in programming not all programs are the same.
For instance, to know that $\mathbb N \to \mathbb N$ is inhabited
it suffices to exhibit any computable total function on the natural numbers.
However, we might be interested in specific such functions, such as factora, Fibonacci, etc.
Moreover, in programming we even go one step further,
since two extensionally equivalent function may be intensionally different,
e.g., smaller/faster programs are preferred. -->

# Polymorphism and implicit arguments

````
module code.id where
````

The polymorphic identity function in Agda is written as follows:

```
id : (A : Set) → A → A
id A x = x
```

(Compare this with the equivalent Haskell definition:
```haskell
id :: A -> A
id x = x
```
Incidentally, note the crucial difference that the typing operator in Agda is `:` instead of `::`.)

Therefore, in Agda `id ℕ` is the identity on the natural numbers,
`id (ℕ → ℕ)` is the identity on functions of natural numbers,
and so on.
It turns out that 1) specifying the argument `A` is boring
and 2) in most cases Agda can infer it from the context.
For these two reasons, Agda allows us to say that some argument is implicit `{A : Set}`, as we demonstrate below.

````
id : {A : Set} → A → A
id x = x
````

**UNDER THE HOOD:** A code cell must start with a line in the format

```agda
module A.B.C where
```

Upon evaluation, the file `A/B/C.agda` is created on disk from the cell's contents
and fed to the Agda interpreter.
For this reason, it is important to evaluate a cell to ensure that its changes are reflected on disk and can be used from other cells.

**SPACING:**
Agda allows us to be very flexible in the variable names,
which can be strings such as `x`, `idλ`, `a→b`, `&&true`, or even arbitrary unicode symbols `💔`, provided there is no space in-between.
As a consequence, spaces in Agda have a syntactic meaning as separators,
and we need to be very generous with them.

## Exercise

Write a polymorphic function `fst` that takes two arguments of types `A` and `B`,
resp., and returns the first argument.

In [ ]:
module code.fst where

fst : {A B : Set} → ?
fst = ?

# Intuitionistic propositional logic in Agda

## Implication

````
module code.impl where
````

Intuitionistic implication is implemented as *function space*
$$A \to B.$$
This concept has no counterpart in classical logic.
The function space operator `→` is an Agda primitive.
The idea is that a proof of $A \to B$
is a (terminating) *program* $t : A \to B$
that, given a proof $a : A$ of $A$,
always terminates and produces a proof $t\; a : B$ of $B$.
The $\to$-eliminaton rule is implemented by *function application*.

````
apply : {A B : Set} → (A → B) → A → B
apply a→b = λ a → a→b a
````

Equivalent spellings:
`apply a→b a = a→b a`,
` apply a→b = a→b`,
`apply = id` (why?).
The last case shows why do not need to explicity use `apply`.

<!--- - The validity problem for implication intuitionistic propositional logic is PSPACE-complete. What is the complexity for the corresponding problem for classical logic? coNP-complete. -->

### **Exercise**

Prove in Agda following tautologies of intuitionistic propositional logic:
1. Argument commutativity: $(A \to B \to C) \to B \to A \to C$.
2. Distributivity: $(A \to B \to C) \to (A \to B) \to A \to C$.
3. Diamond $(A \to B) \to (A \to C) \to (B \to C \to D) \to A \to D$.
4. Projection: $A \to A \to A$. How many proof of this fact are there?

Note that the operator "$\to$" is right-associative:
For example, the last expression above is parenthesized as
$$ A \to (A \to A).$$
We will only write parentheses when they are needed.

Also note that, since intuitionistic implication in general behaves differently from classical implication,
the intuitionistic tautologies above hold for reasons which are different from why the same formulas are also classical tautologies (and thus must be reproven in the inuitionistic setting).

In [ ]:
module code.lab01.ex01 where

→-comm : {A B C : Set} → (A → B → C) → B → A → C
→-comm f b a = ?

→-distr : {A B C : Set} → (A → B → C) → (A → B) → A → C
→-distr f g x = ?

→-diamond : {A B C D : Set} → (A → B) → (A → C) → (B → C → D) → A → D
→-diamond f g h x = ?

→-proj : {A : Set} → A → A → A
→-proj = ?

## Conjunction

````
module code.and where
````

In intuitionistic logic, a proof of $A \wedge B$ is a pair $\langle a, b \rangle$,
where $a$ is a proof of $A$ and $b$ is a proof of $B$.
(From this point of view, intuitionistic conjunction behaves similarly to classical conjunction,
because also in classical logic both conjuncts must be provable for their conjunction to be provable.)
This intuition translates immediately into the following *product* datatype:

````
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
````

The datatype `_∧_` is parametrised by two types, `A` and `B`,
and it is written `A ∧ B` or, using prefix notation, `_∧_ A B`.
It has only one constructor `⟨_,_⟩`, corresponding to the ∧-introduction rule:
Given elements `a : A` and `b : B`,  `⟨ a , b ⟩` has type `A ∧ B` (notice the mandatory spaces!).
The term above can also be written in prefix notation as `⟨_,_⟩ A B`.
We can then define the two projection functions, which correspond to the two ∧-elimination rules:

````
fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b
````

It is sometimes convenient to write longer conjunctions such as `A ∧ B ∧ C`,
to which end we need to define that the operator `_∧_` is *right associative*.

````
infixr 2 _∧_
````

We will also give it a numerical priority, `2` in this case,
in order to avoid ambiguity later then introducing more operators.

Once again, `_∧_ ` is not an Agda primitive, but it can be defined with Agda's datatype creation facility.

### **Exercise**

Formalise and prove the following:
1. Curry/uncurry: $A → B → C$ is the same as $A \wedge B → C$.
2. Conjunction is commutative: $A \wedge B$ is the same as $B \wedge A$.
3. Conjunction is associative: $A \wedge B \wedge C$ is the same as $(A \wedge B) \wedge C$.
4. Implication distributes over conjunction: $A \to B \wedge C$ is the same as $(A \to B) \wedge (A \to C)$.

Do the last three properties follow from curry/uncurry?

In [ ]:
module code.lab01.ex04 where
open import code.and

uncurry : {A B C : Set} → (A → B → C) → A ∧ B → C
uncurry = ?

curry : {A B C : Set} → (A ∧ B → C) → A → B → C
curry = ?

∧-comm : ?
∧-comm = ?

∧-assoc-1 : ?
∧-assoc-1 = ?

∧-assoc-2 : ?
∧-assoc-2 = ?

→∧-distr-1 : ?
→∧-distr-1 = ?

→∧-distr-2 : ?
→∧-distr-2 = ?

## Disjunction

````
module code.or where
````

In intuitionistic logic a proof of a disjunction is a proof of either the first disjunct or of the second.
More precisely, a proof of $A_1 \vee A_2$ is a pair $(k, t_k)$,
where $k \in \{1, 2\}$ specifies that we are proving $A_k$
and $t_k : A_k$ is a proof thereof.
The two options can be implemented with two different constructors,
called right and left *injection*.

````
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B
````

The two constructors `left` and `right` above
correspond to the two ∨-introduction rules.

Notice that this is very different from classical logic
(where it is "easier" to prove a disjunction),
because a classical proof does not need to prove any single disjunct.

We give disjunction lower priority than conjunction
so we can omit parenthesis from `(A ∧ B) ∨ C` and write `A ∧ B ∨ C` instead.

````
infixr 1 _∨_ 
````

The ∨-elimination rule is provided by case analysis,
which is implemented by performing pattern matching on the constructor.


````
case : {A B C : Set} → (A → C) → (B → C) → A ∨ B → C
case f g (left x) = f x
case f g (right x) = g x
````

### **Exercise**

Formalise and prove the following:
1. Disjunction is commutative: $A \vee B$ is the same as $B \vee A$.
2. Disjunction is associative: $A \vee B \vee C$ is the same as $(A \vee B) \vee C$.
3. $A \vee B \to C$ is the same as $(A \to C) \wedge (B \to C)$.

In [ ]:
module code.lab01.ex5 where
open import code.and
open import code.or

∨-comm : {A B : Set} → ?
∨-comm = ?

∨-assoc : {A B C : Set} → ?
∨-assoc = ?

∨∧→-1 : {A B C : Set} → ?
∨∧→-1 = ?

∨∧→-2 : {A B C : Set} → ?
∨∧→-2 = ?

In [ ]:
module code.lab01.ex5 where
open import code.and
open import code.or

∨-comm : {A B : Set} → A ∨ B → B ∨ A
∨-comm (left a) = right a
∨-comm (right a) = left a

∨-assoc : {A B C : Set} → A ∨ B ∨ C → (A ∨ B) ∨ C
∨-assoc (left a) = left (left a)
∨-assoc (right (left b)) = left (right b)
∨-assoc (right (right c)) = right c

∨∧→-1 : {A B C : Set} → (A ∨ B → C) → (A → C) ∧ (B → C)
∨∧→-1 a∨b→c = ⟨ (λ a → a∨b→c (left a)) , (λ b → a∨b→c (right b)) ⟩

∨∧→-2 : {A B C : Set} → (A → C) ∧ (B → C) → A ∨ B → C
∨∧→-2 ⟨ a→c , _ ⟩ (left a) = a→c a
∨∧→-2 ⟨ _ , b→c ⟩ (right b) = b→c b

### **Exercise** (`∧` and `∨`)

Prove the following tautologies:
1. $A \wedge (B \vee C) \leftrightarrow A \wedge B \vee A \wedge C$.
2. $A \vee B \wedge C \leftrightarrow (A \vee B) \wedge (A \vee C)$.

In [ ]:
module code.lab01.distr where
open import code.and
open import code.or

∧∨-distr-1 : {A B C : Set} → A ∧ (B ∨ C) → A ∧ B ∨ A ∧ C
∧∨-distr-1 = ?

∧∨-distr-2 : {A B C : Set} → A ∧ B ∨ A ∧ C → A ∧ (B ∨ C)
∧∨-distr-2 = ?

∨∧-distr-1 : {A B C : Set} → A ∨ B ∧ C → (A ∨ B) ∧ (A ∨ C)
∨∧-distr-1 = ?

∨∧-distr-2 : {A B C : Set} → (A ∨ B) ∧ (A ∨ C) → A ∨ B ∧ C
∨∧-distr-2 = ?

## Truth values

````
module code.true-false where
````

The two truth values `⊤` and `⊥` are implemented via Agda's data type mechanism.
Notice that neither `⊤` nor `⊥` are Agda primitives.
However, Agda's datatype creation facilities allow us to define those datatypes in such a way that they behave as we expect.

### True: `⊤`

The type `⊤` has precisely one inhabitant, called `tt` (its only constructor).

````
data ⊤ : Set where
  tt : ⊤
````

The ⊤-introduction rule says that we can prove that anything implies `⊤`.

````
A→⊤ : {A : Set} → A → ⊤
A→⊤ _ = tt
````

There is no ⊤-elimination rule.

### False: `⊥`

The type `⊥` has dual property to `⊤`.
The type `⊥` has no inhabitants at all, and thus we do not provide any constructor in its definition.

````
data ⊥ : Set where
````

The ⊥-elimination rule says that anything can be proved from `⊥`.
The absurd pattern `()` below is how we tell Agda that there cannot be any argument to `⊥-elim`, and thus it says that no defining equation is needed.

````
⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()
````

There is no ⊥-introducion rule.
This makes the world a better place (why?).

## Negation

````
module code.neg where
open import code.true-false public
````

Negation is not a primitive in intuitionistic logic.
In intuitionistic logic $\neg A$ means that, if we had a proof of $A$, then we could derive a contradiction $\bot$:
$$ \neg A \;\equiv\; A \to \bot.$$
And this is how negation is defined in Agda.

````
¬_ : Set → Set
¬ A = A → ⊥
````

Thus, `¬ A` is a shorthand for (i.e., evaluates to) `A → ⊥`.
Notice that for the first time we are defining a function that maps types to types, i.e., a so called *type-level function*.
This is made possible by the fact that in a dependently typed language
types are first-class citizens and can be manipulated as any other data.

We give negation higher priority than `∧` and `∨`,
so we can just write `¬ A ∧ B` instead of `(¬ A) ∧ B.

````
infix 3 ¬_ 
````

### **Exercise** (`¬¬`)

The logic of Agda is intuitionistic. In particular, in Agda the following double negation law does *not* hold:
$$ A \leftrightarrow \neg \neg A. $$
Which one of the two directions holds in intuitionistic logic?
- Formalise this and prove it in Agda.
- Does the proof (i.e., program) resemble something we have already seen?

*Hint:* The type $\neg \neg A$ expands to
$$(A \to \bot) \to \bot.$$

In [ ]:
module code.lab01.ex02 where
open import code.neg

-- recall that ¬ ¬ A = (A → ⊥) → ⊥

¬¬-intro : {A : Set} → A → ¬ ¬ A
¬¬-intro x f = ?

-- the occurrence of "?" above is called a hole;
-- Agda will compile and remind us that there are open goals corresponding to holes to be solved

### **Exercise** (`¬ B → ¬ A`)

The *contrapositive* of an implication $A \to B$ is $\neg B \to \neg A$.
In classical logic an implication and its contrapositive are logically equivalent, i.e., the following is a tautology:
$$(A \to B) \leftrightarrow (\neg B \to \neg A).$$
Use Agda to prove which, if any, of the two directions above holds in intuitionistic logic.

In [ ]:
module code.lab01.ex03 where
open import code.neg

-- your solution here
contrapositive : ?
contrapositive = ?

### **Exercise**  (De Morgan laws)

Are the following laws valid in intuitionistic logic?
If so, write a proof in Agda.
\begin{align}
 (1) \qquad \neg (A \vee B) \leftrightarrow \neg A \wedge \neg B. \\
 (2) \qquad \neg A \vee \neg B \to \neg (A \wedge B). \\
 (3) \qquad \neg (A \wedge B) \to \neg A \vee \neg B.
\end{align}

In [ ]:
module code.lab01.ex07 where
open import code.and
open import code.or
open import code.neg

de_morgan1-1 : {A B : Set} → ¬ (A ∨ B) → ¬ A ∧ ¬ B
de_morgan1-1 = ?

de_morgan1-2 : {A B : Set} → ¬ A ∧ ¬ B → ¬ (A ∨ B)
de_morgan1-2 = ?

de_morgan2 : {A B : Set} → ¬ A ∨ ¬ B → ¬ (A ∧ B)
de_morgan2 = ?

de_morgan3 : {A B : Set} → ¬ (A ∧ B) → ¬ A ∨ ¬ B
de_morgan3 = ?

### **Exercise** (`¬ A ∨ B`)

<!-- (Classical vs. intuitionistic implication) -->

In classical logic, $A \to B$ is defined to be $\neg A \vee B$.
Which of the following two directions hold in intuitionistic logic? Prove it in Agda.
$$(A \to B) \leftrightarrow (\neg A \vee B).$$

<!--- *Hint:* Classical implication intuitionistically implies intuitionistic implication. For this reason, classical logic can be seen as a conservative extension of intuitionistic logic. -->

In [ ]:
module code.lab01.ex07 where
open import code.true-false
open import code.neg
open import code.or

-- your solution here

# Intuitionistic first-order  logic

We show how dependent types can be used to implement universal quantification (dependent function space)
and existential quantification (dependent product),
thus concluding our overview of intuitionistic logic.

````
module code.universal where
````

## The ∀ quantifier
In intuitionistic logic a proof of $\forall (a : A) B$ is a function $f$ mapping a proof $a$ of $A$
into a proof $f\; a$ of $B\; a$, where we can see $B$ as a family of types indexed by proofs of $A$.
The universal quantifier is implemented via the *dependent function space*

````
Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a

-- compare with implication:
-- ((a : A) → B a) versus A → B
````

This generalises implication `A → B`, which corresponds to non-dependent function space.
In this sense, in intuitionistic logic implication is a special case of universal quantification.

Universal quantification also generalises conjunction,
since the type $B_1 \wedge B_2$ is isomorphic to $\Pi\; A\; B$
where $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, sometimes `Π` is called dependent product (hence the notation).

````
-- the type of the first argument of Π can be inferred from the second
forAll : {A : Set} → (B : A → Set) → Set
forAll {A} B = Π A B

-- we introduce a convenient syntax reminiscent of universal quantification in logic
∀-syntax = forAll
infix 0 ∀-syntax
syntax ∀-syntax (λ x → B) = ∀[ x ] B

-- dependent apply; corresponds to ∀-elimination
apply : {A : Set} → {B : A → Set} → Π A B → (a : A) → B a
apply f x = f x
````

### An example

````
open import code.nat.even-odd
open import code.or
````

We show an application of universal quantification
and prove that every natural number is either even or odd:

````
even∨odd : ∀[ n ] Even n ∨ Odd n
````

We now explain how the syntax above works.
The latter expands to 

```agda
-- even∨odd : forAll (λ n → Even n ∨ Odd n)
```

which in turn is the same as `Π A B`
with `A = ℕ` and `B = λ n → Even n ∨ Odd n`
(we can avoid typing `n` because its occurrence in `Even n` implies `n : ℕ`), i.e.,

```agda
-- even∨odd : Π ℕ (λ n → Even n ∨ Odd n)
```

By unrolling the definition of `Π A B = (a : A) → B a`,
we finally get

```agda
-- even∨odd : (n : ℕ) → Even n ∨ Odd n
```

This shows precisely how the universal quantifier is implemented by having `B a` to depend on `(a : A)`.
Thus, while `∀[ n ] Even n ∨ Odd n` is just syntactic sugaring for the more familiar `(n : ℕ) → Even n ∨ Odd n`,
we will use the former when we want to emphasise its logical content,
and the latter when we want to emphasise its computational content.
As a courtesy of the Curry-Howard isomorphism, they correspond to each other.

We now proceed to prove `even∨odd`,
which is another instance of external verification.
The proof is by induction on `n`.
In the base case `n = 0` and we can use the base constructor `zero : Even zero`
(notice how `zero` is overloaded):

````
even∨odd zero = left zero
````

In the inductive case, we proceed as follows:

````
even∨odd (suc n) with even∨odd n
... | left even = right (suc even)
... | right odd = left (suc odd)
````

### The `with` pattern matching construct

In order to decide whether `suc n` is even or odd,
we recursively call `even∨odd n` and inspect its result:
if it pattern matches with `left even` (where `even : Even n`)
then we return `right (suc even)` (where `suc even : Odd (suc n)`, again `suc` is overloaded),
and symmetrically in the other case.
This is the first time that we see the `with` construct,
which allows us to extend the set of arguments that we can pattern match on.

### **Exercise** (`∀`, `∧`, and `∨`)

Which of the following are intuitionistic tautologies?
Which are classic tautologies?
Which are none?
Prove the intuitionistic ones.

1. $\forall a B \wedge C \to (\forall a B) \wedge (\forall a C)$.
2. $(\forall a B) \wedge (\forall a C) \to \forall a B \wedge C$.
3. $\forall a B \vee C \to (\forall a B) \vee (\forall a C)$.
4. $ \forall a B \vee C \to (\forall a B) \vee C$, where $a$ does not occurr in $C$.
5. $(\forall a B) \vee (\forall a C) \to \forall a (B \vee C)$.

*Hint:* If you cannot easily program a solution, then most likely there is no solution.

In [ ]:
module code.universal.ex1 where
open import code.universal
open import code.and
open import code.or

-- 1.
∀∧-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∧ C a → (∀[ a ] B a) ∧ (∀[ a ] C a)
∀∧-distr = ?

-- 2.
∧∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∧ (∀[ a ] C a) → ∀[ a ] B a ∧ C a
∧∀-distr = ?

-- 3.
∀∨-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∨ C a → (∀[ a ] B a) ∨ (∀[ a ] C a)
∀∨-distr = ?

-- 4.
∀∨-distr' : {A C : Set} {B : A → Set} → ∀[ a ] B a ∨ C → (∀[ a ] B a) ∨ C
∀∨-distr' = ?

-- 5.
∨∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∨ (∀[ a ] C a) → ∀[ a ] B a ∨ C a
∨∀-distr = ?

## The existential quantifier

````
module code.existential where
open import code.universal
open import code.eq
````

In intuitionistic logic,
a proof of $\exists (a : A) B$ is a pair $(a, b)$,
where $a$ is a proof of $A$
and $b$ is a proof of $B\; a$.
Like in universal quantification, we can see $B$ as a family of types indexed by proofs of $A$.
The existential quantifier is implemented with the *dependent product*:

````
data Σ (A : Set) (B : A → Set) : Set where
    ⟨_,_⟩ : (a : A) → B a → Σ A B
````

Compare this with conjunction, which corresponds to non-dependent product $A \wedge B$:

```agda
-- data _∧_ (A : Set) (B : Set) : Set where
--  ⟨_,_⟩ : A → B → A ∧ B
```

In this sense, in intuitionistic logic existential quantification `Σ` generalises conjunction,
which justifies the name dependent product.
(This can create confusion because `Π` is sometimes called dependent product too,
since also `Π` generalises conjunction.)

Existential quantification also generalises disjunction,
since the type $B_1 \vee B_2$ is isomorphic to $\Sigma\; A\; B$
with $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, `Σ` is sometimes called dependent sum.

````
thereExists : ∀ {A : Set} (B : A → Set) → Set
thereExists {A} B = Σ A B

∃-syntax = thereExists
infix 0 ∃-syntax
syntax ∃-syntax (λ x → B) = ∃[ x ] B

-- aka uncurry
∃-elim : {A : Set} {B : A → Set} {C : Set} → ((a : A) → B a → C) → Σ A B → C
∃-elim a→b→c ⟨ a , b ⟩ = a→b→c a b
````

### **Exercise** (`∃`, `∧`, and `∨`)

Establish whether the following are intuitionistic tautologies
and prove it in Agda for the positive cases:

1. $\exists a B \vee C \to (\exists a B) \vee (\exists a C)$.
2. $(\exists a B) \vee (\exists a C) \to \exists a B \vee C$.
3. $\exists a B \wedge C \to (\exists a B) \wedge (\exists a C)$.
4. $(\exists a B) \wedge (\exists a C) \to \exists a B \wedge C$.
5. $(\exists a B) \wedge C \to \exists a B \wedge C$, where $a$ does not occurr in $C$.

In [ ]:
module code.existential.prop where
open import code.existential
open import code.and
open import code.or

-- 1
∃∨-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∨ C a → (∃[ a ] B a) ∨ (∃[ a ] C a)
∃∨-distr = ?

-- 2
∨∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∨ (∃[ a ] C a) → ∃[ a ] B a ∨ C a
∨∃-distr = ?

-- 3
∃∧-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∧ C a → (∃[ a ] B a) ∧ (∃[ a ] C a)
∃∧-distr = ?

-- 4
∧∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∧ (∃[ a ] C a) → ∃[ a ] B a ∧ C a
∧∃-distr = ?

-- 5
∧∃-distr' : {A : Set} {B : A → Set} {C : Set} → (∃[ a ] B a) ∧ C → ∃[ a ] B a ∧ C
∧∃-distr' = ?

### **Exercise** (`∀`, `∃`, and `¬`)

 Which of the following hold in intuitionistic logic? Prove it.

1. $\exists a \forall b C \to \forall b \exists a C$, where $C$ depends on $a$ and $b$.
2. $\exists a \neg B \to \neg \forall a B$, where $B$ depends on $a$.
3. $\neg \forall a B \to \exists a \neg B$, where $B$ depends on $a$.

In [ ]:
module code.universal-existential where
open import code.universal
open import code.existential
open import code.neg

-- 1
-- B cannot depend on (a : A) for the swap to be possible!
∃∀-distr : {A : Set} {B : Set} {C : A → B → Set} → ∃[ a ] ∀[ b ] C a b → ∀[ b ] ∃[ a ] C a b
∃∀-distr = ?

-- 2
¬∃→∀¬ : {A : Set} {B : A → Set} → ∃[ a ] ¬ B a → ¬ (∀[ a ] B a)
¬∃→∀¬ = ?

-- 3
¬∀→∃¬ : {A : Set} {B : A → Set} → ¬ (∀[ a ] B a) → ∃[ a ] ¬ B a
¬∀→∃¬ = ?

# Challenges

In [ ]:
module code.conn where
open import code.or public
open import code.neg public 
open import code.and public 
open import code.true-false public

## **Exercise** (Triple negation)

While $\neg \neg A \to A$ does not hold in intuitionistic logic, prove that the following *triple negation* law holds

$$\neg \neg \neg A \to \neg A.$$

*Hint:* Expand the definition of "$\neg$". You should get a function of two arguments and output $\bot$.

In [ ]:
module code.lab01.ex03 where
open import code.conn

¬¬¬-rule : {A : Set} → ¬ ¬ ¬ A → ¬ A
¬¬¬-rule = ?

## **Exercise** (Irrefutability)

<!--- In classical logic we have the following *law of excluded middle*:
$$ A \vee \neg A. $$
Why there is no Agda program of the corresponding type `{A : Set} → A ∨ ¬ A`? -->

Show that the following classical tautologies $P$ are intuitionistically *irrefutable*, in the sense that $\neg \neg P$ is an intuitionistic tautology:

1. Law of excluded middle: $\neg \neg (A \vee \neg A)$.
   *Hint: Expand the definition of $\neg$. You will need: `left`, `right`, and $\lambda$-abstraction.*
2. Implication as disjunction: $\neg \neg ((A \to B) \to \neg A \vee B)$.
3. De Morgan: $\neg \neg (\neg (A \wedge B) \to \neg A \vee \neg B)$.

In [ ]:
module code.lab01.irref where
open import code.conn

excluded-middle-irref : ?
excluded-middle-irref = ?

impl-irref : ?
impl-irref = ?

deMorgan-irref : ?
deMorgan-irref = ?

## **Exercise** (Weak Peirce's law)

Prove the following weakening of Peirce's law:
$$((((A \to B) \to A) \to A) \to B) \to B.$$

In [ ]:
module code.lab01.weak-peirce where

wp : {A B : Set} → ((((A → B) → A) → A) → B) → B
wp = ?

## **Exercise**

In the previous exercises we have seen that the following principles are not intuitionistic tautologies:
1. Law of excluded middle: $A \vee \neg A$.
2. Elimination of double negation: $\neg \neg A \to A$.
3. Implication as disjunction: $(A \to B) \to \neg A \vee B$.
4. The Negated De Morgan's law: $\neg (\neg A \wedge \neg B) \to A \vee B$.
5. Peirce's Law: $((A \to B) \to A) \to A$.

Show that all principles above are logically equivalent in intuitionistic logic.
Each propositional variable $A, B$ is universally quantified in each principle.

*Hint:* Prove the following sequence of implications:
- $1 \to 2$.
- $2 \to 3$: Use irrefutability of $(A → B) \to \neg A \vee B$, proved earlier:
$ \neg \neg ((A \to B) \to \neg A \vee B).$
- $3 \to 1$.
- $1 \to 4$: Use the excluded middle for $A$ and for $B$.
- $4 \to 1$: Use $\neg (\neg A \wedge \neg B) \to A \vee B$ with $B \equiv \neg A$.
- $1 \to 5$.
- $5 \to 1$: Use Peirce's law $((A' \to B') \to A') \to A'$ with
$A' \equiv A \vee \neg A$ and $B' \equiv \bot.$

In [ ]:
module code.lab01.LEM-eq where
open import code.conn public
open import code.lab01.irref public

1→2 : ?
1→2 = ?

2→3 : ?
2→3 = ?

3→1 : ?
3→1 = ?

1→4 : ?
1→4 = ?

4→1 : ?
4→1 = ?

1→5 : ?
1→5 = ?

5→1 : ?
5→1 = ?

## **Exercise**

Show that the following two principles are intuitionistically equivalent:
1. De Morgan's Law: $\neg (A \wedge B) \to \neg A \vee \neg B$.
2. The *weak principle of excluded middle*: $\neg A \vee \neg \neg A$.
This is interesting, because the weak principle of excluded middle is strictly weaker than the principle of excluded middle, but it is still not an intuitonistic tautology.

In [ ]:
module code.lab01.weak-LEM where

1→2 : ?
1→2 = ?

2→1 : ?
2→1 = ?